In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import random
import os

class GirlDataset(Dataset):
    def __init__(self, data_path, transform):
        self.data_path = data_path
        self.transform = transform
        self.dataset = []
        self.preprocess()
 
    def preprocess(self):
        member_list = ['yoona', 'taeyeon', 'tiffany']
        folder_list = os.listdir(self.data_path)
        for folder in folder_list:
          if folder in member_list:
            label = folder.split('/')[-1]
            if label == 'yoona':
              label = 0
            elif label == 'taeyeon':
              label = 1
            elif label == 'tiffany':
              label = 2
            file_list = os.listdir(os.path.join(self.data_path, folder))
            for file in file_list:
              data = os.path.join(self.data_path, folder, file)
              if data[-3:] == 'png':
                self.dataset.append([data, label])
        
        print('Processing Finish')
        
    def __getitem__(self, idx):
        filename, label = self.dataset[idx]
        image = Image.open(filename)
        image = self.transform(image)
        return image, label
    
    def __len__(self):
        return len(self.dataset)
    

In [ ]:
from torchvision import transforms as T

crop_size=178
image_size=128
transform = []
transform.append(T.RandomHorizontalFlip())
transform.append(T.Resize(image_size))
transform.append(T.ToTensor())
transform.append(T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)))
transform = T.Compose(transform)

In [ ]:
train_dataset = GirlDataset(data_path = '/gdrive/My Drive/러닝스푼즈/data/GirlGroup/processed/',
                      transform = transform)

train_loader = DataLoader(dataset=train_dataset,
                         batch_size=64,
                         shuffle=True)

test_dataset = GirlDataset(data_path = '/gdrive/My Drive/러닝스푼즈/data/GirlGroup/test/',
                      transform = transform)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=64,
                         shuffle=False)

Processing Finish
Processing Finish


In [ ]:
import torch.nn as nn

class ConvNet(nn.Module):
    def __init__(self, num_classes=3):
      super(ConvNet, self).__init__()
      self.layer1 = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
      self.layer2 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
      self.fc = nn.Linear(32768, num_classes)
        
    def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = out.reshape(out.size(0), -1)
      out = self.fc(out)
      return out

32768

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
learning_rate = 0.001
num_epochs = 1000
# Loss and optimizer
model = ConvNet(num_classes=3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_loss = []
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
  model.train()
  for i, batch in enumerate(train_loader):
    images, labels = batch[0], batch[1]
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    
    loss = criterion(outputs, labels)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    total_loss.append(loss.item())

  print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
          .format(epoch+1, num_epochs, i+1, total_step, sum(total_loss)/len(total_loss)))
  
  model.eval()
  with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(test_loader):
      images, labels = batch[0], batch[1]
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)

      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the 12 test images: {} %'.format(100 * correct / total))
      

Epoch [1/1000], Step [6/6], Loss: 10.5011
Test Accuracy of the model on the 12 test images: 33.333333333333336 %
Epoch [2/1000], Step [6/6], Loss: 7.3596
Test Accuracy of the model on the 12 test images: 33.333333333333336 %
Epoch [3/1000], Step [6/6], Loss: 6.4402
Test Accuracy of the model on the 12 test images: 33.333333333333336 %
Epoch [4/1000], Step [6/6], Loss: 5.6894
Test Accuracy of the model on the 12 test images: 41.666666666666664 %
Epoch [5/1000], Step [6/6], Loss: 5.2468
Test Accuracy of the model on the 12 test images: 41.666666666666664 %
Epoch [6/1000], Step [6/6], Loss: 4.7819
Test Accuracy of the model on the 12 test images: 41.666666666666664 %
Epoch [7/1000], Step [6/6], Loss: 4.5549
Test Accuracy of the model on the 12 test images: 33.333333333333336 %
Epoch [8/1000], Step [6/6], Loss: 4.2044
Test Accuracy of the model on the 12 test images: 41.666666666666664 %
Epoch [9/1000], Step [6/6], Loss: 3.9637
Test Accuracy of the model on the 12 test images: 25.0 %
Epoch